In [1]:
pip install panel


     ---------------------------------------- 0.0/20.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/20.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/20.8 MB ? eta -:--:--
     --------------------------------------- 0.0/20.8 MB 393.8 kB/s eta 0:00:53
     --------------------------------------- 0.1/20.8 MB 459.5 kB/s eta 0:00:45
     --------------------------------------- 0.1/20.8 MB 504.4 kB/s eta 0:00:41
     --------------------------------------- 0.1/20.8 MB 532.5 kB/s eta 0:00:39
     --------------------------------------- 0.2/20.8 MB 655.1 kB/s eta 0:00:32
     --------------------------------------- 0.2/20.8 MB 686.6 kB/s eta 0:00:30
     --------------------------------------- 0.3/20.8 MB 630.2 kB/s eta 0:00:33
      -------------------------------------- 0.3/20.8 MB 679.5 kB/s eta 0:00:31
      -------------------------------------- 0.4/20.8 MB 696.3 kB/s eta 0:00:30
      -------------------------------------- 0.4/20.8 MB 713.5 k

In [2]:
import os 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


In [3]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

In [4]:
file_input = pn.widgets.FileInput(width=300)

openaikey = pn.widgets.PasswordInput(
    value="", placeholder="Enter your OpenAI API Key here...", width=300
)
prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type', 
    options=['stuff', 'map_reduce', "refine", "map_rerank"]
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings", margin=10
    ), width=600
)

In [5]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [6]:
convos = []  # store all panel objects in a list

def qa_result(_):
    os.environ["OPENAI_API_KEY"] = openaikey.value
    
    # save pdf file to a temp file 
    if file_input.value is not None:
        file_input.save("/.cache/temp.pdf")
    
        prompt_text = prompt.value
        if prompt_text:
            result = qa(file="/.cache/temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
            convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
            #return convos
    return pn.Column(*convos, margin=15, width=575, min_height=400)


In [7]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

In [8]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=630, scroll=True)

In [9]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F60A! Question Answering with your PDF file
    
    1) Upload a PDF. 2) Enter OpenAI API key. This costs $. Set up billing at [OpenAI](https://platform.openai.com/account). 3) Type a question and click "Run".
    
    """),
    pn.Row(file_input,openaikey),
    output,
    widgets

).servable()

Traceback (most recent call last):
 File "C:\Anaconda\conda\lib\site-packages\pyviz_comms\__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "C:\Anaconda\conda\lib\site-packages\panel\viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Anaconda\conda\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\util\callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Anaconda\conda\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Anaconda\conda\lib\site-packages\bokeh\util\callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Anaconda\conda\lib\site-packages\panel\reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "C:\Anaconda\conda\lib\site-packages\panel\io\state.py", line 408, in _handle_exception
 raise exception
 File "C:\Anaconda\conda\lib\site-packages\panel\reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Anaconda\conda\lib\site-packages\panel\reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "C:\Anaconda\conda\lib\site-packages\panel\widgets\button.py", line 183, in _process_event
 self.param.trigger('value')
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 1993, in trigger
 self_.set_param(**dict(params, **triggers))
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 1929, in set_param
 return self_.update(kwargs)
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Anaconda\conda\lib\site-packages\panel\param.py", line 838, in _replace_pane
 new_object = self.eval(self.object)
 File "C:\Anaconda\conda\lib\site-packages\panel\param.py", line 818, in eval
 return function(*args, **kwargs)
 File "C:\Anaconda\conda\lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "C:\Anaconda\conda\lib\site-packages\panel\depends.py", line 209, in wrapped
 return function(*combined_args, **combined_kwargs)
 File "C:\Users\HP\AppData\Local\Temp\ipykernel_3836\920887063.py", line 8, in qa_result
 file_input.save("/.cache/temp.pdf")
 File "C:\Anaconda\conda\lib\site-packages\panel\widgets\input.py", line 214, in save
 with open(fn, 'wb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/.cache/temp.pdf'
 


 Traceback (mos

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Row(sizing_mode='stretch_width')
        [0] FileInput(width=300)
        [1] PasswordInput(placeholder='Enter your OpenAI A..., width=300)
    [2] WidgetBox(scroll=True, width=630)
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] ParamFunction(function, _pane=Column, loading_indicator=True, sizing_mode='stretch_width')
    [3] Row(width=600)
        [0] Column(margin=5, sizing_mode='stretch_width')
            [0] TextEditor(height=160, placeholder='Enter your questions h..., sizing_mode='stretch_width', toolbar=False)
            [1] Button(name='Run!', sizing_mode='stretch_width')
        [1] Card(margin=10, sizing_mode='stretch_width', title='Advanced settings')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Chain type', options=['stuff', 'map_reduce', ...], sizing_mode='stretch_width', value='stuff')
                [1] IntSlider(end=5, name='Number of relevant c..., sizing_mode='stretch_width', start=1, value=2)